In [1]:
import pandas as pd
import numpy as np
import wrangle_zillow
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler

In [2]:
# 2017 properties
# keep all the features
# only include last transaction (no duplicate property IDs)
# zestimate error
# data of transaction
# only include properties that have a latitude and longitude

#url = get_url("zillow")

sql = """
select * from properties_2017
join predictions_2017 using(parcelid)
left join airconditioningtype using(airconditioningtypeid)
left join architecturalstyletype using(architecturalstyletypeid)
left join buildingclasstype using(buildingclasstypeid)
left join heatingorsystemtype using(heatingorsystemtypeid)
left join propertylandusetype using(propertylandusetypeid)
left join storytype using(storytypeid)
left join typeconstructiontype using(typeconstructiontypeid)
where latitude IS NOT NULL
and longitude IS NOT NULL
"""

In [3]:
# 1. Acquire data from mall_customers.customers in mysql database.
df = wrangle_zillow.get_zillow_data()

In [4]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [5]:
df.shape

(77580, 69)

In [6]:
# Get the ncounts of all the nulls and sort but greatest to least
null_counts = df.isnull().sum().sort_values(ascending = False)
null_counts.head(25)

buildingclassdesc           77565
buildingclasstypeid         77565
finishedsquarefeet13        77538
storytypeid                 77530
basementsqft                77530
storydesc                   77530
yardbuildingsqft26          77510
fireplaceflag               77408
architecturalstyledesc      77373
architecturalstyletypeid    77373
typeconstructiondesc        77357
typeconstructiontypeid      77357
finishedsquarefeet6         77194
pooltypeid10                77115
decktypeid                  76966
poolsizesum                 76711
pooltypeid2                 76506
hashottuborspa              76041
yardbuildingsqft17          75187
taxdelinquencyyear          74680
taxdelinquencyflag          74680
finishedsquarefeet15        74553
finishedsquarefeet50        71543
finishedfloor1squarefeet    71543
fireplacecnt                69291
dtype: int64

In [7]:
null_counts.tail(25)

yearbuilt                       270
censustractandblock             247
calculatedfinishedsquarefeet    201
structuretaxvaluedollarcnt      115
regionidzip                      50
taxamount                         5
landtaxvaluedollarcnt             2
taxvaluedollarcnt                 1
fips                              0
id                                0
propertylandusedesc               0
assessmentyear                    0
propertylandusetypeid             0
roomcnt                           0
transactiondate                   0
regionidcounty                    0
bathroomcnt                       0
logerror                          0
bedroomcnt                        0
rawcensustractandblock            0
propertycountylandusecode         0
id.1                              0
longitude                         0
latitude                          0
parcelid                          0
dtype: int64

In [8]:
#Figuring out how much data is missing and where
def nulls_by_cols(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing/rows
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_missing_rows': pct_missing})
    return cols_missing

In [9]:
# inplace = True modifies the original df.
nulls_by_column = nulls_by_cols(df)
nulls_by_column.sort_values(by='percent_missing_rows', ascending=False, inplace=True)
nulls_by_column.head()

,number_missing_rows,percent_missing_rows
buildingclassdesc,77565,0.999807
buildingclasstypeid,77565,0.999807
finishedsquarefeet13,77538,0.999459
storytypeid,77530,0.999356
storydesc,77530,0.999356


In [10]:
# axis=1 for columns
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing= pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing})\
    .reset_index().groupby(['num_cols_missing','pct_cols_missing'])\
    .count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

nulls_rows = nulls_by_row(df)
nulls_rows.sort_values(by='pct_cols_missing', ascending=False, inplace=True)
nulls_rows.head(15)

,num_cols_missing,pct_cols_missing,num_rows
25,48,69.56521739130434,3
24,47,68.11594202898551,3
23,46,66.66666666666666,5
22,45,65.21739130434783,50
21,44,63.76811594202898,79
20,43,62.31884057971014,29
19,42,60.86956521739131,27
18,41,59.42028985507246,29
17,40,57.971014492753625,230
16,39,56.52173913043478,285


## Prepare Goals
- How do we define a single unit? No land/lot, duplexes, commercial
- Remove any properties that are likely something other than single unit.
    - Do not purely filter by unitcnt
    - Goal is to reduce single units that are falsely removed as something else.
- Ideas:
    - filter out 0 bedroom and 0 bathroom properties.
    - room count greater than 1
    - squarefootage more than 400
    - filter by propertylandusetype
    - keep:
    - 260, residential general
    - 261, single family residential
    - 262, rural residence
    - 279 inferred single family residential

In [11]:
# Use the isin function to get all properties having the values described above
df = df[df.propertylandusetypeid.isin([260,261,262,279])]
df.shape
# Went from 77580 to 52484

(52484, 69)

In [12]:
# filtering 0 bedroom and 0 bathroom properties
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]
df.shape

(52300, 69)

In [13]:
df.unitcnt.value_counts()

1.0    33783
2.0       28
4.0        3
3.0        1
Name: unitcnt, dtype: int64

In [14]:
# We have over 18K null units. They are probably unit counts of 1. 
# It's more likely the paperwork for a single unit didn't have a value
df.unitcnt.isnull().sum()

18485

In [15]:
df.unitcnt = df.unitcnt.fillna(1)
df.shape

(52300, 69)

In [18]:
# How many unitcnt are not 1?
(df.unitcnt != 1).sum()
#Shows 32 that slipped by that could be duplex or other type of units

32

In [19]:
# Filter the duplex, triplex, etc...
df = df[df.unitcnt == 1.0]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52268 entries, 0 to 77579
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52268 non-null  int64  
 1   typeconstructiontypeid        76 non-null     float64
 2   storytypeid                   47 non-null     float64
 3   propertylandusetypeid         52268 non-null  float64
 4   heatingorsystemtypeid         33899 non-null  float64
 5   buildingclasstypeid           0 non-null      float64
 6   architecturalstyletypeid      70 non-null     float64
 7   airconditioningtypeid         13627 non-null  float64
 8   id                            52268 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52268 non-null  float64
 11  bedroomcnt                    52268 non-null  float64
 12  buildingqualitytypeid         33697 non-null  float64
 13  c

In [22]:
# Drop logically unnecessary columns
# id
# id.1
df = df.drop(columns=["id", "id.1"])

In [23]:
def handle_missing_values(df, prop_required_column = .60, prop_required_row = .60):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [24]:
df.shape

(52268, 67)

In [25]:
df = handle_missing_values(df)

In [26]:
df.shape

(52268, 33)

In [28]:
# Columns to drop b/c they're ids of descriptions
# propertylandusetypeid
# heatingorsystemtypeid
df = df.drop(columns=["propertylandusetypeid", "heatingorsystemtypeid"])

In [29]:
df.isnull().sum().sort_values(ascending = False)

buildingqualitytypeid           18571
propertyzoningdesc              18510
heatingorsystemdesc             18369
regionidcity                     1029
lotsizesquarefeet                 351
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         71
yearbuilt                          40
regionidzip                        23
calculatedbathnbr                  16
fullbathcnt                        16
calculatedfinishedsquarefeet        8
taxamount                           4
landtaxvaluedollarcnt               1
taxvaluedollarcnt                   1
latitude                            0
bedroomcnt                          0
fips                                0
bathroomcnt                         0
propertylandusedesc                 0
longitude                           0
propertycountylandusecode           0
rawcensustractandblock              0
regionidcounty                      0
roomcnt                             0
unitcnt     

In [31]:
# propertyzoningdesc = Description of the allowed land uses (zoning) for that property
# I'll drop this, b/c we're already filtering for single unit residential.
df = df.drop(columns =['propertyzoningdesc'])

In [32]:
df.heatingorsystemdesc.isnull().sum()

18369

In [33]:
df.heatingorsystemdesc.value_counts()

Central       20723
Floor/Wall    12542
Forced air      517
Solar            85
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [34]:
# Because we are dealing with properties in southern california, they do not need a heater
# We will fill with none
df.heatingorsystemdesc = df.heatingorsystemdesc.fillna("None")

In [39]:
# Almost equivalent to each other so we can drop calculatedbathnbr
(df.calculatedbathnbr == df.bathroomcnt).sum()/len(df)

0.9996938853600673

In [41]:
df = df.drop(columns=['calculatedbathnbr'])

In [43]:
df.isna().sum().sort_values(ascending=False).head(10)

buildingqualitytypeid           18571
regionidcity                     1029
lotsizesquarefeet                 351
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         71
yearbuilt                          40
regionidzip                        23
fullbathcnt                        16
calculatedfinishedsquarefeet        8
dtype: int64

## It's a Good Time to Split the Data
- Split the df into train/validate/test
- Impute values from train, then apply them to validate and test sets
- Keep our out of sample datasets out of sample

In [44]:
train_and_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_and_validate, test_size=.3, random_state=123)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52268 entries, 0 to 77579
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52268 non-null  int64  
 1   bathroomcnt                   52268 non-null  float64
 2   bedroomcnt                    52268 non-null  float64
 3   buildingqualitytypeid         33697 non-null  float64
 4   calculatedfinishedsquarefeet  52260 non-null  float64
 5   finishedsquarefeet12          52102 non-null  float64
 6   fips                          52268 non-null  float64
 7   fullbathcnt                   52252 non-null  float64
 8   latitude                      52268 non-null  float64
 9   longitude                     52268 non-null  float64
 10  lotsizesquarefeet             51917 non-null  float64
 11  propertycountylandusecode     52268 non-null  object 
 12  rawcensustractandblock        52268 non-null  float64
 13  r

In [46]:
# Categorical/Discrete columns to use mode to replace nulls. Create a list of the categorical variables to
# replace the nulls

cols = [
    "buildingqualitytypeid",
    "regionidcity",
    "regionidzip",
    "yearbuilt",
    "regionidcity",
    "censustractandblock"
]

for col in cols:
    mode = int(train[col].mode()) # I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [48]:
train.isnull().sum().sort_values(ascending=False).head()

lotsizesquarefeet               183
finishedsquarefeet12             92
structuretaxvaluedollarcnt       37
fullbathcnt                       8
calculatedfinishedsquarefeet      4
dtype: int64

In [50]:
cols = [
    "structuretaxvaluedollarcnt",
    "taxamount",
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "finishedsquarefeet12",
    "calculatedfinishedsquarefeet",
    "fullbathcnt",
    "lotsizesquarefeet"
]


for col in cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [57]:
train.isnull().sum().sort_values(ascending=False).sum()

0

In [58]:
validate.isnull().sum().sort_values(ascending=False).sum()

0

In [59]:
test.isnull().sum().sort_values(ascending=False).sum()

0

## Then Write The Output to Disk and Move Forward

In [60]:
train.to_csv("zillow_train.csv")
validate.to_csv("zillow_validate.csv")
test.to_csv("zillow_test.csv")